In [2]:
import sys
sys.path.append('../python')
from load_generate_data import load_generate_data
from parse_go_tree import parese_go_tree
import numpy as np
from env import *
import csv
import tensorflow as tf
from sklearn import preprocessing
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier, plot_tree
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter
import graphviz 
from load_generate_data import *
from random import shuffle

1_GSE150430_B cell [BTC000020]
get data
get gene ontology structer
(1800, 18763)


In [102]:
def get_sub_matrix_cell_type(mt, cell_type_name, barcodes):
    cell_types = np.array([name.split(']')[0].split('[')[-1] for name in barcodes])
    indices = np.where(cell_types == cell_type_name)[0]
    sub_mt = mt[indices, :]
    pct = sum(sub_mt > 0.5, 1) / sub_mt.shape[0]
    return np.where(pct >= 0.5)[0]

def get_valid_col(mt):
    pct = sum(mt > 0.5, 1) / mt.shape[0]
    return np.where(pct <= 0.5)[0]

def select_col(X, Y_origin):
    cell_types = list(map(lambda x: x.split(']')[0].split('[')[-1], Y_origin))
    cell_types = np.unique(cell_types)
    valid_col = get_valid_col(X)
    ans = set()
    for cell_type in cell_types:
        cell_type_main = get_sub_matrix_cell_type(X, cell_type, Y_origin)
        valid = list(set(valid_col) & set(cell_type_main))
        ans = ans | set(valid)
    return np.array(list(ans))
    
print(len(selected_col))

5938


In [51]:
def get_all_data(list_study_path):
    data = None
    have_data = False
    barcodes = []
    for study in list_study_path:
        mt, barcode_study = load_generate_data(study)
        print(mt.shape)
        if have_data == False:
            have_data = True
            data = mt
        else:
            data = np.concatenate((data, mt), axis=0)
        barcodes.extend(barcode_study)
    data = np.array(data)

    barcode_origin = np.array([('_').join(x.split('_')[1:]) for x in barcodes])
    barcodes = [x.split('_')[-1] for x in barcodes]
    le = preprocessing.LabelEncoder()                                                                                                                                         
    le.fit(barcodes)                                                                                                                                                        
    barcodes = le.transform(barcodes)
    return data, barcodes, barcode_origin

def split_data(X, Y, Y_origin):
    number_class = max(Y)
    print("Number of class is", Y)
    print(Counter(list(map(lambda x: x.split(']')[0].split('[')[-1], Y_origin))))
    train_indices = set()
    test_indices = set()
    for i in range(number_class):
        pos = np.where(Y == i)[0]
        # Random arr
        shuffle(pos)
        divide_position = int(len(pos) * 0.7)
        train_indices = train_indices | pos[0:divide_postion]
        test_indices = test_indices | pos[divide_postion:]
    return np.array(list(train_indices)), np.array(list(test_indices))

list_study_path =   ['/home/tung/RepresentData/Transform/GSE110686',
                    '/home/tung/RepresentData/Transform/GSE124310',
                    '/home/tung/RepresentData/Transform/GSE150430']

X, Y, Y_origin = get_all_data(list_study_path)
train_indices, test_indices = split_data(X, Y, Y_origin)

(500, 18763)
(1800, 18763)
(1800, 18763)


In [ ]:
clf = tree.DecisionTreeClassifier(max_depth=20, max_leaf_nodes= max(Y) * 2)
clf = clf.fit(X[train_indices, :], Y[train_indices])
print("Score :", clf.score(X[train_indices, :], Y[train_indices]))
print("Test :", clf.score(X[test_indices, :], Y[test_indices]))


In [52]:
dot_data = tree.export_graphviz(clf, out_file=None) 
graph = graphviz.Source(dot_data) 
graph.render("t") 
# kmeans = KMeans(n_clusters=50, random_state=0).fit(data)
# group = np.array(kmeans.labels_)

# for i in range(max(group) + 1):
#     list_index = np.where(group == i)
#     print(Counter(barcode_origin[list_index]))
#     print("########################################")
#     print("########################################")
#     print("########################################")
#     print("########################################")

[ 1  1  1 ... 20 20 20]
